<a href="https://colab.research.google.com/github/yosshor/jupyter_notebook_projects/blob/master/GPT_Images_project_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Bot Name 'nayabigdataengineer'


In [1]:
!pip install -q openai
!pip install -q gradio
!pip install pyTelegramBotAPI
!pip install Google-Images-Search
!pip install SpeechRecognition
!pip install gTTS
!pip3 install pydub
!pip install pocketsphinx
!pip install pymongo

     |████████████████████████████████| 44 kB 2.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 147 kB 15.1 MB/s 
     |████████████████████████████████| 13.8 MB 6.7 MB/s 
     |████████████████████████████████| 278 kB 58.8 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 86 kB 4.8 MB/s 
     |████████████████████████████████| 2.3 MB 65.0 MB/s 
     |████████████████████████████████| 55 kB 4.8 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 106 kB 84.7 MB/s 
     |████████████████████████████████| 64 kB 3.3 MB/s 
     |████████████████████████████████| 80 kB 11.0 MB/s 
     |████████████████████████████████| 69 kB 6.4 MB/s 
     |████████████████████████████████| 58 kB 7.4 MB/s 
     |████████████████████████████████| 50 kB 8.1 MB/s 
Looking in indexes: htt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 1.2 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 4.6 MB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.1 MB 1.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import openai
import telebot
from telebot import types
import time
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import os
import re
from google_images_search import GoogleImagesSearch
import nltk
import random
import pprint
from nltk import Tree
import speech_recognition as sr
from gtts import gTTS, lang
from os import path
from pydub import AudioSegment  
from pymongo import MongoClient
import string
from numpy.random import seed
from numpy.random import randint
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
from email import encoders

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
!curl ipecho.net/plain

34.125.100.66

In [7]:
telid=[]
with open('telegram_test_bot_token.txt','r') as f:
     telid.append(f.read().split('\n'))

In [37]:
test_token = telid[0][0].split('telegramBot:')[1]
openai.api_key = telid[0][1].split('gpt_api:')[1]
api_google_image_key = telid[0][2].split('googleSearchImage:')[1]
your_project_cx = telid[0][3].split('=')[1]
mongo_connecting_str = telid[0][4]
email_autenticate = telid[0][5]

In [35]:

bot_url = f'https://api.telegram.org/bot{test_token}/'  
url = bot_url + f'getUpdates'
resp = requests.get(url)
message = json.loads(resp.text)['result']
chat_id = message[0]['message']['from']['id'] #= 319184502
username = message[0]['message']['from']['first_name'] + " " +message[0]['message']['from']['last_name']

In [45]:
def openai_chat(prompt):
    completions = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        temperature=0.5,
    )

    message = completions.choices[0].text
    return message.strip()

In [48]:

patterns = """
    NP: {<JJ>*<NN*>+}
    {<JJ>*<NN*><CC>*<NN*>+}
    """

NPChunker = nltk.RegexpParser(patterns)

def prepare_text(input):
    sentences = nltk.sent_tokenize(input)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    sentences = [NPChunker.parse(sent) for sent in sentences]
    return sentences


def parsed_text_to_NP(sentences):
    nps = []
    for sent in sentences:
        tree = NPChunker.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'NP':
                t = subtree
                t = ' '.join(word for word, tag in t.leaves())
                nps.append(t)
    return nps


def sent_parse(input):
    sentences = prepare_text(input)
    nps = parsed_text_to_NP(sentences)
    return nps

def find_nps(text):
    prepared = prepare_text(text)
    parsed = parsed_text_to_NP(prepared)
    final = sent_parse(parsed)

In [49]:
def parse_the_recipe_name(message):
  sent = str(re.split(":|\.|\n|\!", message)[0])
  matches = []
  for i in sent.split():
    if i[0].isupper():
      regex = "^[A-Z]\w*$" #"^\s*(?:\b[A-Z]+\b[\s]*)+(?:[:-])\s*$" #
      matches.append(re.findall(regex, i))
      print(i)
  flat_list = [item for sublist in matches[1:] for item in sublist]

  recipe_name = ' '.join(map(str,flat_list))
  print(recipe_name)
  return recipe_name

def parse_the_recipe_name_new(message):
  sent = str(re.split(":|\.|\n|\!", message)[0])
  print(sent_parse(sent))
  print(sent_parse(sent)[-1])
  return sent_parse(sent)[-1]


In [50]:
def make_dir(folder_name):
  if not os.path.exists(folder_name):
      os.mkdir(folder_name)
      return True
  return False

In [51]:
def download_images(recipe_name,path_to_download,chat_id,file_type):
  gis = GoogleImagesSearch(api_google_image_key, your_project_cx)
  _search_params = {
      'q': f'{recipe_name}',
      'num':1,
      'imagesize':'500x500', #large
      'fileType': f'{file_type}' } #jpg|gif|
  # search first, then download and resize afterwards:
  gis.search(search_params=_search_params,  width=500, height=500)
  for image in gis.results():
      image.url  # image direct url
      print(image.url) 
      image.referrer_url  # image referrer url (source)   
      print(image.referrer_url) 
      image.download(path_to_download)  # download image
      image.path  # downloaded local file path
      print(f'image path : {image.path}')
      bot.send_photo(chat_id, photo=open(image.path, 'rb'))

  print('all the images downloaded successfully')

In [52]:
def save_result(path,messasge_to_write):
  with open(path, "w") as text_file:
    text_file.write(messasge_to_write)
  print('File saved')
  return

In [53]:
def speech_to_text(filename):
  # initialize the recognizer
  r = sr.Recognizer()
  # open the file
  with sr.AudioFile(filename) as source:
      # listen for the data (load audio to memory)
      audio_data = r.record(source)
      # recognize (convert from speech to text)
      text = r.recognize_google(audio_data)
      print(text)
      return text

In [54]:
def speechToText(file_name):
                                                                        
    src = f"{file_name}"
    dst = f"{file_name}.wav"
    # convert wav to mp3     
    # https://pythonbasics.org/convert-mp3-to-wav/                                                       
    sound = AudioSegment.from_ogg(src)
    sound.export(dst, format="wav")
    # Initialize the recognizer  
    r = sr.Recognizer()  
    AUDIO_FILE = f"{file_name}.wav"
    with sr.AudioFile(AUDIO_FILE) as source:
        audio2 = r.record(source)  # read the entire audio file
        MyText = r.recognize_sphinx(audio2) 
        MyText = MyText.lower() 
        print(MyText)
        return MyText
    return -1

In [56]:
def send_email(fromwho, to, subject, text,path_folder, files=None):
    assert isinstance(to, list)
    print("inside the email function")
    msg = MIMEMultipart()
    msg['From'] = str(fromwho) 
    msg['To'] = COMMASPACE.join(to)
    msg['Date'] = formatdate(localtime=True)
    msg['Subject'] = subject
    msg.attach(MIMEText(text))
    for file in files:
        combined_file = path_folder +'/' + file
        print(combined_file)
        part = MIMEBase('application', "octet-stream")
        part.set_payload(open(combined_file, "rb").read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(combined_file))
        msg.attach(part)
    fromaddr = fromwho
    smtp = smtplib.SMTP('smtp.gmail.com', 587)
    smtp.starttls()
    smtp.login(fromaddr, email_autenticate)
    text = msg.as_string()
    smtp.sendmail(fromwho, to, msg.as_string())
    smtp.close()

In [57]:
def convert_to_speach(text_to_convert,file_path,file_name,language = 'en'):
  myobj = gTTS(text=text_to_convert, lang=language, slow=False)
  myobj.save(f"{file_path}/{file_name}.mp3")
  os.system(f"mpg321 {file_path}/{file_name}.mp3")
  return

In [58]:
def insert_to_DB(mongo_connecting_str,user_name,chat_id_mongo,user_email,type_wanted,json_result,query):
    client = MongoClient(mongo_connecting_str);
    print("Connection Successful")
    collection_name =  client['Yos'].get_collection('NayaProject')
    item_1 = {
            "user_name" : user_name,
            "chat_id" : chat_id_mongo,
            "user_email" : user_email,
            "type_wanted" : type_wanted,
            "result" : json_result,
            "query":query
          }
    collection_name.insert_one(item_1)
    print("inserted succussfully")
    return

In [59]:
def get_results_and_image(query,root_dir,message_chat_id,type_wanted,mongo_connecting_str):
   resu = str(openai_chat(query))
   print(type(resu))
   print(resu)
   recipe_name = parse_the_recipe_name(resu)
   print(sent_parse(resu))
   if recipe_name == "" :
     recipe_name = query.split(' ')[-1]
     print("inside the false")
   print(recipe_name)
   path = os.path.join(root_dir, recipe_name)
   recipe_dir = make_dir(os.path.join(root_dir, recipe_name))
   print(recipe_dir)
   print(f"trying to save the result into {path}/{recipe_name}.txt")
   save_result(path+f'/{recipe_name}.txt',resu)
   from_email ='******@gmail.com'

   if(type_wanted == "Audio" ):
     convert_to_speach(resu,path,recipe_name,language = 'en')
     print(f"{path}/{recipe_name}.mp3")
     bot.send_audio(chat_id=message_chat_id, audio=open(f"{path}/{recipe_name}.mp3", 'rb'))
   else:
      bot.send_message(message_chat_id,str(replace_for_result(resu)) ,parse_mode="MarkdownV2")

  
   try:
     download_images(recipe_name,path,message_chat_id,'png')
     list_of_files_to_send = os.listdir(path)
     if user_email != '' :
       try:
          print(list_of_files_to_send)
          send_email(from_email,[user_email], query, 'Here is the result for your query', path, list_of_files_to_send)
          bot.send_message(message_chat_id, f'the files sent succussfully to your mail')

       except Exception as e :
          bot.send_message(message_chat_id, f'some error occured : {str(e)}')   
     chat_id_mongo = str(message_chat_id)
     user_email_add = user_email if user_email != '' else "user_email"
     insert_to_DB(mongo_connecting_str,username,chat_id_mongo,user_email_add,type_wanted,f"{{'result':'{resu}}}",query)
     bot.send_message(message_chat_id, f'inserted succussfully to MongoDB')

   except Exception as e:
     bot.send_message(message_chat_id, f'cant insert it to MongoDB The Error is: {e}')
   return


In [60]:
%rm -rf '/content/Recipes'
%mkdir '/content/Recipes'

#### 'how to make icecream' 
#### 'how to make pasta'
#### 'how to make salad'
#### which recipe can i do with tomatoes and cucumbers and cheese
#### which recipe can i do with water tomatoes and cheese
####how to make coffee
####how to make greek salad


In [70]:


def replace_for_result(some_str):
  return some_str.replace('.', '\.').replace('=', '\=').replace('{', '\{').replace('}', '\}').replace('(', '\(').replace(')', '\)').replace('-', '\-').replace('`', '').replace('*', '').replace('#', '\#').replace('!', '\!')

root_dir = "/content/Recipes"



bot = telebot.TeleBot(test_token)

@bot.message_handler(content_types=['voice'])
def voice_processing(message):
    file_info = bot.get_file(message.voice.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    file_name = f'{random.randint(100000,999999)}.ogg'
    with open(file_name, 'wb') as new_file:
        new_file.write(downloaded_file)
    bot.reply_to(message, "File saved")
    print(f'/content/{file_name.split(".")[0]}')
    text_from_voice = speechToText(f'/content/{file_name}') 
    # text_from_voice = speech_to_text('/content/new_file.ogg') 
    bot.reply_to(message, f'You Said : {text_from_voice}')


@bot.message_handler(commands=['help'])
def help(message):
     bot.reply_to(message,"""Available Commands :-
    /email - insert your email address 
    /data_to_excel - For getting all the data from mongoDB in excel file""")


@bot.message_handler(commands=['gif'])
def voice_processing(message):
   bot.reply_to(message, f'gif : {message.text.split("/gif")[1:]}')
   dir = '/content/new'
   query = ' '.join(message.text.split('/gif')[1:]).lstrip()
   download_images(query,dir,message.chat.id,'gif')

@bot.message_handler(commands=['start'])
def start(message):
    bot.reply_to(message,"""Available Commands :-
    /email - insert your email address 
    /data_to_excel - getting all data from mongoDB in excel.
    /myemail - what is my email address
    """)


@bot.message_handler(commands=['email'])
def get_email(message):
    global user_email 
    print(message.text)
    user_email = message.text.split(' ')[1]
    print(user_email)
    bot.reply_to(message, f'Your Email : {user_email}')


@bot.message_handler(commands=['myemail'])
def get_email(message):
    try :
        bot.reply_to(message, f'Your Email : {user_email}')
    except Exception as error:
          bot.reply_to(message, 'Please insert your email address')


#dont forget to add your ip into mongoDB
@bot.message_handler(commands=['data_to_excel'])
def get_excel(message):
    random_name =''.join(random.choices(string.ascii_lowercase, k=10))
    client = MongoClient(mongo_connecting_str);
    collection_name =  client['Yos'].get_collection('NayaProject')
    item_details = collection_name.find()
    pd.DataFrame(item_details).to_excel(f"{random_name}.xlsx")
    document = open( f"/content/{random_name}.xlsx", 'rb')
    bot.send_document(message.chat.id, document)


def makeKeyboard():
    markup = types.ReplyKeyboardMarkup()
    markup.add(types.InlineKeyboardButton(text="Text",callback_data="Text"),
    types.InlineKeyboardButton(text="Audio",callback_data="Audio"))
    return markup 


@bot.message_handler(content_types=['text'])
def handle_command_adminwindow(message):
  global query 
  global message_chat_id
  global recipe_name

  if message.text == "Audio":
      print(message.text)
      print(query)
      bot.send_chat_action(message_chat_id, 'typing')
      get_results_and_image(query,root_dir,message_chat_id,"Audio",mongo_connecting_str)
      return

  elif message.text == "Text":
       print(message.text)
       bot.send_chat_action(message_chat_id, 'typing')
       get_results_and_image(query,root_dir,message_chat_id,"Text",mongo_connecting_str)
       return
  else :
      bot.send_message(chat_id=message.chat.id,
                text="How Do You Want Your Result Format ?",
                reply_markup=makeKeyboard(),
                parse_mode='HTML') 
      query = message.text
      message_chat_id = message.chat.id
      print(query)
bot.polling()



how to make greek salad
Text
<class 'str'>
1. Start by prepping your vegetables. Chop the cucumber, red onion, and tomatoes into small cubes. Slice the olives and bell pepper into thin slices.

2. In a large bowl, combine the cucumber, red onion, tomatoes, olives, bell pepper, and feta cheese.

3. In a small bowl, whisk together the olive oil, red wine vinegar, oregano, garlic powder, salt, and pepper.

4. Pour the dressing over the vegetables and toss to combine.

5. Serve the Greek salad with a sprinkle of oregano and a few slices of lemon, if desired. Enjoy!

['Start', 'cucumber', 'red onion', 'large bowl', 'cucumber', 'red onion', 'pepper', 'feta cheese', 'small bowl', 'whisk', 'olive oil', 'red wine vinegar', 'oregano', 'garlic powder', 'salt', 'pepper', 'dressing', 'toss', 'Greek salad', 'sprinkle', 'oregano', 'lemon', 'Enjoy']
inside the false
salad
False
trying to save the result into /content/Recipes/salad/salad.txt
File saved
https://images.getrecipekit.com/20210728051512-IG%

In [ ]:
client = MongoClient(mongo_connecting_str);
collection_name =  client['Yos'].get_collection('NayaProject')
 
item_details = collection_name.find()
for item in item_details:
   print(item)